In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x
# Define the dataset and data loader
train_folder = 'D:/Data/Train'
test_folder = 'D:/Data/Test'
transform = transforms.Compose([
    transforms.Resize((480,640)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transforms.Compose([
    transforms.Resize((480, 640)),
    transforms.ToTensor()
]))

train_loader = DataLoader(train_dataset, batch_size=9600, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=9600, shuffle=False)

# Instantiate the neural network and define the loss function and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
for epoch in range(15):
    model.train()  # Set the model to training mode
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        scores = model(data)
        loss = criterion(scores, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print training progress
        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{15}], Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Evaluate the neural network on the test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            scores = model(data)
            _, predicted = torch.max(scores.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')



In [ ]:
torch.save(model.state_dict(), 'person_detection_model_v3_74.pth')
print('Модель создана')

In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Определение архитектуры нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Загрузка весов модели
model = Net()
model.load_state_dict(torch.load('D:/Курсовой проект/Diploma project (Дилом)/src/CNN model/person_detection_model_v3_74.pth'))
model.eval()

# Создание примерного входного тензора
dummy_input = torch.randn(1, 3, 32, 32)

# Конвертация модели в формат ONNX
onnx_path = 'person_detection_model.onnx'
torch.onnx.export(model, dummy_input, onnx_path, opset_version=11)

print('Модель успешно сконвертирована в формат ONNX и сохранена в файле person_detection_model.onnx.')


In [ ]:
import cv2
print(cv2.__version__)

In [ ]:
import torch
import torch.onnx as onnx

# Шаг 1: Загрузка модели PyTorch
model = torch.load("D:/Курсовой проект/Diploma project (Дилом)/src/CNN model/person_detection_model_v3_74.pth")  # Путь к вашей модели PyTorch

# Шаг 2: Создание объекта ONNX и настройка его на использование CPU или GPU
# Определение примерного входного размера (batch_size, channels, height, width)
input_size = (1, 3, 32, 32)

# Шаг 3: Преобразование модели PyTorch в ONNX
dummy_input = torch.randn(*input_size)  # Создание примерного входного тензора
input_names = ["input"]
output_names = ["output"]
onnx_path = "model.onnx"  # Путь для сохранения модели ONNX

# Шаг 4: Сохранение модели ONNX на диск
torch.onnx.export(model, dummy_input, onnx_path, input_names=input_names, output_names=output_names)


In [ ]:
import torch
import torchvision
import torch.onnx

# Загрузка предобученной модели Faster R-CNN с ResNet50-FPN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Заморозка параметров всех слоев
for param in model.parameters():
    param.requires_grad = False

# Замена классификатора модели на классификатор, специфичный для распознавания только людей
in_features = model.roi_heads.box_predictor.cls_score.in_features
num_classes = 2  # 1 класс для фона и 1 класс для людей
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Создание примера входных данных
input_data = torch.randn(1, 3, 480, 640)  # Пример входных данных размером 1x3x224x224

# Установка имен входных и выходных тензоров
input_names = ["input"]
output_names = ["output"]

# Экспорт модели в формат ONNX
torch.onnx.export(model,
                  input_data,
                  "model.onnx",
                  input_names=input_names,
                  output_names=output_names,
                  opset_version=11)

# Сохранение модели ONNX на диск
torch.onnx.save(model, "model.onnx")


In [ ]:
import onnx

# Загрузка модели Faster R-CNN из файла ONNX
model = onnx.load("D:/Курсовой проект/Diploma project (Дилом)/src/OPNX_FPGA/model.onnx")
# Check that the model is well formed
onnx.checker.check_model(model)
# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

In [ ]:
import torch
import torchvision

# Загрузите предварительно обученную модель Faster R-CNN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Экспортируйте модель в ONNX формат
torch.onnx.export(model, torch.randn(1, 3, 224, 224), 'fasterrcnn_resnet50_fpn.onnx', opset_version=11)


In [3]:
import torch
import torch.nn as nn
from torchviz import make_dot

# Определение модели нейронной сети
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# Создание экземпляра модели
model = Net()

# Создание фиктивных входных данных
dummy_input = torch.randn(1, 3, 32, 32)

# Выполнение прямого прохода по модели
output = model(dummy_input)

# Создание графа
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render("neural_network_architecture")


ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH